## In this file, i want to properly determine dN/dy by fitting an exponential only on the low pt part : then extrapolate it to low pt and determine dNdy and <pT> with the associated uncertainties that are well described

In [61]:
import ROOT as r
import numpy as np
import math 
import matplotlib.pyplot as plt

In [62]:
PbPb_pp_file = r.TFile("ROOTfile/PbPb_pp5tev.root")

In [63]:
Histo = PbPb_pp_file.Get("Table 5")
#Histo.ls()

In [64]:
TGraphAsymm = Histo.Get("Graph1D_y1;1")
Y1 = Histo.Get("Hist1D_y1;1")

In [65]:
## Now i pay attention to the error that are put into the error bin. In this table, they are 3 different errors

e1 = Histo.Get("Hist1D_y1_e1")  # statistics
e2 = Histo.Get("Hist1D_y1_e2")  # correlated systematics
e3 = Histo.Get("Hist1D_y1_e3")  # uncorrelated systematics

# I define new values for this graph, where i quadratically sum the uncorrelated errors

n = TGraphAsymm.GetN()

for i in range(n):

    err = math.sqrt(e1.GetBinContent(i+1)**2 + e3.GetBinContent(i+1)**2)

    exl = TGraphAsymm.GetErrorXlow(i)
    exh = TGraphAsymm.GetErrorXhigh(i)

    TGraphAsymm.SetPointError(i, exl, exh, err, err)

In [66]:
## Exponential function

## Here the range is way more smaller and focus on low-pt

fit_min = 0.3 
fit_max = 2.5

m_p = 0.938272 #mass of the proton in Gev/c²

def exp_fct(x, par):
    
    pt = x[0]
    dNdy = par[0]
    T = par[1]
    mp = par[2]

    mt = math.sqrt(pt*pt+mp*mp)

    Factor = dNdy * pt * (1/(T*(mp + T))) 

    Expression = math.exp(-(mt-mp)/T)

    return Factor * Expression


In [67]:
exp = r.TF1("exp1", exp_fct, fit_min, fit_max, 3)

In [68]:
#exp parameters

exp.SetParNames("dNdy", "T", "mp")

exp.SetParameters(10, 0.35, 0.938272)

exp.FixParameter(2, 0.938272)


In [69]:
fit_exp = TGraphAsymm.Fit(exp,"RS0")

****************************************
Minimizer is Minuit2 / Migrad
Chi2                      =      54.4797
NDf                       =           25
Edm                       =  1.96576e-06
NCalls                    =           80
dNdy                      =      86.8471   +/-   1.31542     
T                         =     0.763502   +/-   0.0119954   
mp                        =     0.938272                      	 (fixed)


In [70]:
# Plot of the exponential fit

canva_exp = r.TCanvas("canvabis", "comparaison")
canva_exp.SetGrid()
canva_exp.SetLogy()

TGraphAsymm.SetTitle("Fit of the low p_{T} part by exponential function")

TGraphAsymm.GetXaxis().SetTitle("p_{T}[Gev/c]")
TGraphAsymm.GetYaxis().SetTitle("(1/Nev)*D²(N)/DptDyrap [(Gev/c^{⁻1}]")
TGraphAsymm.GetXaxis().SetRangeUser(0., 10.)
TGraphAsymm.SetMinimum(1e-2)

exp.SetLineColor("kRed")
exp.SetLineStyle(1)   

TGraphAsymm.Draw("AP")     
exp.Draw("same")

legend = r.TLegend(0.60,0.68,0.88,0.88)

legend.SetTextFont(42)       
legend.SetTextSize(0.035)   
legend.SetMargin(0.25)       

legend.AddEntry(exp, "exponential", "l")
legend.AddEntry(TGraphAsymm, "p#bar{p} data", "l")


CHI = str(exp.GetChisquare())[:5]
NDF = str(exp.GetNDF())

Text_exp = r.TPaveText(4, 10, 6, 30)
Text_exp.AddText(f" #chi^{{2}} = {CHI}")
Text_exp.AddText(f" NdF = {NDF} ")
Text_exp.Draw()

print(fit_exp)

legend.Draw()

canva_exp.Draw()


****************************************
Minimizer is Minuit2 / Migrad
Chi2                      =      54.4797
NDf                       =           25
Edm                       =  1.96576e-06
NCalls                    =           80
dNdy                      =      86.8471   +/-   1.31542     
T                         =     0.763502   +/-   0.0119954   
mp                        =     0.938272                      	 (fixed)



Warning in <TCanvas::Constructor>: Deleting canvas with same name: canvabis


### I will determine dNdy and <pt> in parralel :

##### dN/dy in this fit correspond to the "area" of the data. 
The strategy is to extrapolate the fit on the low pT part (0-3 Gev/c) and calculate the integral. In the measured range (0.3-20 Gev/c), we sum the bins area.


#####  for pT : pT_mean = Int(pT * f(pT) * dpT) / Int(f(pT) * dpT) 
In the extrapolate part I define a function that is pT times the fitting function, and I integrate it. It gives the numerator that I call J_external. For the measured part I multiply by pT (the center of the bin) the area and the sum gives an estimation of J_measure.

In [71]:
# Value of the integral on the extrapolated part

exp.SetRange(0.0, fit_max)
I_extrapolated =exp.Integral(0.0, fit_min)


# Define of the pT * f(pT) function that is used to compute <pT>

g = r.TF1("g", lambda x, p: x[0]*exp.EvalPar(x, p), 0, fit_max, exp.GetNpar())

for i in range(exp.GetNpar()):
    g.SetParameter(i, exp.GetParameter(i))

J_ext = g.Integral(0.0, fit_min) ## J_external is the denominator of pt mean in the external part



In [72]:
#Here I create a function that adapt the uncertainties only taking the uncorrelated one

def adapt_error(TF1, err1, err2, err3):


    for i in range(1, TF1.GetNbinsX()+1): #nb of bins
        err = math.sqrt( err1.GetBinContent(i)**2 +  err3.GetBinContent(i)**2 )  ##We didn't take now into account correlated uncertainties now
        TF1.SetBinError(i, err) 
    

adapt_error(Y1, e1, e2, e3)

In [73]:
## This is the main function of this programm. It compute dNdy, J and the uncertainties associated (stat and syst uncorrelated)

def dNdy_compute(TF1, err_stat, err_syst): ## Here we put uncorrelated uncertainties, 

    dndy_estimate = 0
    low_uncertainties_stat = 0
    up_uncertainties_stat = 0
    low_uncertainties_syst = 0
    up_uncertainties_syst = 0

    bin_area = 0
    low_error_syst = 0
    up_error_syst = 0
    low_error_stat = 0
    up_error_stat = 0

    Jmeas =  0
    sigmaJ_meas_square_stat = 0
    sigmaJ_meas_square_syst = 0
    pt = 0
    

    for i in range(1, TF1.GetNbinsX()+1):

        pt = TF1.GetXaxis().GetBinCenter(i)      ## pT at the center of the bin
        bin_area = TF1.GetBinContent(i) * TF1.GetBinWidth(i)       ##  y * delta(pT) 


        low_error_stat = err_stat.GetBinContent(i) * TF1.GetBinWidth(i)    ## Here the uncertainties are symmetrics so i can get the content directly
        up_error_stat = err_stat.GetBinContent(i) * TF1.GetBinWidth(i) 

        low_error_syst = err_syst.GetBinContent(i) * TF1.GetBinWidth(i) 
        up_error_syst = err_syst.GetBinContent(i) * TF1.GetBinWidth(i)

        low_uncertainties_stat = low_uncertainties_stat + low_error_stat**2    #sum the lowests uncertainties
        up_uncertainties_stat = up_uncertainties_stat + up_error_stat**2    #sum the upper uncertainties

        low_uncertainties_syst = low_uncertainties_syst + low_error_syst**2   
        up_uncertainties_syst = up_uncertainties_syst + up_error_syst**2 

        sigmaJ_meas_square_stat = sigmaJ_meas_square_stat + (pt * up_error_stat)**2
        sigmaJ_meas_square_syst = sigmaJ_meas_square_syst + (pt * up_error_syst)**2

        
        dndy_estimate = dndy_estimate + bin_area  #This part sum the area of each bin --> dN/dy measured
        Jmeas = Jmeas + pt * bin_area  ## Determination of J_meas


    return dndy_estimate, math.sqrt(up_uncertainties_stat), math.sqrt(low_uncertainties_stat), math.sqrt(up_uncertainties_syst), math.sqrt(low_uncertainties_syst), Jmeas, math.sqrt(sigmaJ_meas_square_stat), math.sqrt(sigmaJ_meas_square_syst)



In [74]:
# I compute 

dndy_mes = dNdy_compute(Y1, e1, e3)

I_meas = dndy_mes[0]  ## The value of the bins area summed
sigma_stat_meas = dndy_mes[1]  ## uncertainties stat and syst
sigma_syst_meas = dndy_mes[3]  

J_meas = dndy_mes[5]
sigmaJ_meas_stat = dndy_mes[6]
sigmaJ_meas_syst = dndy_mes[7]

## We get the value of dN/dy with uncertainties 

print(f"dN/dy_mes = {I_meas:.3f} ± {sigma_stat_meas:.3f} (stat) ± {sigma_syst_meas:.3f} (syst)")



dN/dy_mes = 72.392 ± 0.065 (stat) ± 0.397 (syst)


In [75]:
## Here i calculate the value of <pT>

Itot = I_meas + I_extrapolated
Jtot = J_meas + J_ext

pt_mean = Jtot/Itot

print(pt_mean)

1.435737886704272


##### Now i want to get access to the uncertainties on the integral I extrapolate. For this we are calculating the propagation on the integral of the uncertainties from the parameters of the fit

Source of the method : https://root-forum.cern.ch/t/how-to-use-integralerror/44724 

In [76]:
## So now for I and J 

sigmaI_ext = exp.IntegralError( 0.0, fit_min, fit_exp.GetParams(), fit_exp.GetCovarianceMatrix().GetMatrixArray())

sigmaJ_ext = g.IntegralError( 0.0, fit_min, fit_exp.GetParams(), fit_exp.GetCovarianceMatrix().GetMatrixArray())  ## remind : g is the pT * fit(pT) function


In [77]:
### Now we need to take into account correlated uncertainties. The method of the ALICE collaboration is to shift the point of data, 
# fit again and calculate the integral (total) like we did. Then we substract the two integrals (shifted and unshiffted) and it gave 
# us the correlated uncertainties on integral. 

### Steps

"""Create a new clone of TH1 and TGraph ;
Shift the value of the point ;
Fit again with an exponential ;
Calculate again the value of integrals ;
and finaly take the difference as the correlated uncertainties on the calculation ."""


'Create a new clone of TH1 and TGraph ;\nShift the value of the point ;\nFit again with an exponential ;\nCalculate again the value of integrals ;\nand finaly take the difference as the correlated uncertainties on the calculation .'

In [78]:
# Create a new clone of TH1 and TGraph

TGraphAsymm_shift = Histo.Get("Graph1D_y1;1")
Y1_shift = Histo.Get("Hist1D_y1;1")


In [79]:
# This part assign to the TGraph the new value shifted of the point 

n = TGraphAsymm_shift.GetN()


for i in range(n):

    err = math.sqrt(e1.GetBinContent(i+1)**2 + e3.GetBinContent(i+1)**2)

    exl = TGraphAsymm_shift.GetErrorXlow(i)
    exh = TGraphAsymm_shift.GetErrorXhigh(i)

    TGraphAsymm_shift.SetPointError(i, exl, exh, err, err)

    point = TGraphAsymm_shift.GetPointY(i)
    
    TGraphAsymm_shift.SetPointY(i, point + e2.GetBinContent(i+1))


In [80]:
## I shift by putting the correlated systematics the every bin value

def adapt_error_shift(TF1, err1, err2, err3): 

    for i in range(1, TF1.GetNbinsX()+1): #nb of bins
        
        err = math.sqrt( err1.GetBinContent(i)**2 +  err3.GetBinContent(i)**2 )  ##We didn't take now into account correlated uncertainties now
        TF1.SetBinError(i, err) 
        value = TF1.GetBinContent(i)
        TF1.SetBinContent(i, value + err2.GetBinContent(i) )

adapt_error_shift(Y1_shift, e1, e2, e3)


In [81]:
exp_shift = r.TF1("exp1", exp_fct, fit_min, fit_max, 3)

#exp parameters

exp_shift.SetParNames("dNdy", "T", "mp")

exp_shift.SetParameters(10, 0.35, 0.938272)

exp_shift.FixParameter(2, 0.938272)



In [82]:
## I fit a second time but with the fitted data

fit_exp_shift = TGraphAsymm_shift.Fit(exp_shift,"RS0")

****************************************
Minimizer is Minuit2 / Migrad
Chi2                      =      69.3067
NDf                       =           25
Edm                       =  7.55167e-07
NCalls                    =           79
dNdy                      =      89.7354   +/-   1.33219     
T                         =      0.74703   +/-   0.0116325   
mp                        =     0.938272                      	 (fixed)


In [83]:
## I extrapolate again and determine again the value of I_extrapolate, J_extrapolate

exp_shift.SetRange(0.0, fit_max)
I_err_extra_extrapolated = exp_shift.Integral(0.0, fit_min)

g_shift = r.TF1("g_shift", lambda x, p: x[0]*exp_shift.EvalPar(x, p), 0, fit_max, exp_shift.GetNpar())

for i in range(exp_shift.GetNpar()):
    g_shift.SetParameter(i, exp_shift.GetParameter(i))

J_ext_shift = g_shift.Integral(0.0, fit_min)



In [84]:
# Compute the dNdy from the measure shift part

dNdy_shift = dNdy_compute(Y1_shift, e1, e2)

I_meas_shift = dNdy_shift[0]
J_meas_shift = dNdy_shift[5]

I_tot_shift = I_meas_shift + I_err_extra_extrapolated
J_tot_shift = J_meas_shift + J_ext_shift

pt_shift = J_tot_shift/I_tot_shift



In [85]:
Ishift_tot = I_meas_shift + I_err_extra_extrapolated

## Like described above, the correlated systematics is the value after shift - nominal value, this is true for the integral but also for <pT>

sigma_corr = Ishift_tot-Itot


In [86]:
## To recover the full systematic uncertainty, i sum quadratically the uncorrelated and correlated

sigma_syst = math.sqrt(sigma_corr**2 + dndy_mes[3]**2)
sigma_stat = math.sqrt(sigmaI_ext**2 + dndy_mes[1]**2)

print(f"The statistical uncertainty is {str(sigma_stat)[:5]}, the systematic one is {str(sigma_syst)[:5]}")

The statistical uncertainty is 0.073, the systematic one is 3.678


In [87]:
## This is the final value of dN/dy with all the uncertainties taken in account except the one that depends on the model used 
# (because I fit only with an exponential form)

print(f"dN/dy = {Itot:.3f} ± {float(str(sigma_stat)[:5]):.3f} (stat) ± {float(str(sigma_syst)[:5]):.3f} (syst)")

dN/dy = 75.308 ± 0.073 (stat) ± 3.678 (syst)


In [88]:
# <pT> = Int(pT * f(pT) * dpT) / Int(f(pT) * dpT)

## Modifed exponential function ---> pT * exp

sigmaI_stat_pt = math.sqrt(sigmaI_ext**2 + sigma_stat_meas**2) 
sigmaJ_stat_pt = math.sqrt(sigmaJ_ext**2 + sigmaJ_meas_stat**2) 

#propagate to sigma_stat <pT> (= J/I)

sigma_stat_pt = math.sqrt( (sigmaJ_stat_pt/Itot)**2 + ((Jtot * sigmaI_stat_pt)/((Itot)**2))**2 )
sigma_syst_uncorr_pt = math.sqrt( (sigmaJ_meas_syst/Itot)**2 + ((Jtot * sigma_syst_meas)/((Itot)**2))**2 )

sigma_syst_corr_pt = np.abs(pt_mean-pt_shift)
sigma_syst_pt = math.sqrt( sigma_syst_uncorr_pt**2 + sigma_syst_corr_pt**2)

In [89]:
print(f"dN/dy = {Itot:.3f} ± {float(str(sigma_stat)[:5]):.3f} (stat) ± {float(str(sigma_syst)[:5]):.3f} (syst)")
print(f"<pT> = {pt_mean:.3f} ± {float(str(sigma_stat_pt)[:5]):.3f} (stat) ± {float(str(sigma_syst_pt)[:5]):.3f} (syst)")

dN/dy = 75.308 ± 0.073 (stat) ± 3.678 (syst)
<pT> = 1.436 ± 0.002 (stat) ± 0.014 (syst)


In [90]:
## Here these computations were for the most central collision. Now we'll do a loop to get all these parameters for every centrality